In [1]:
import re

In [2]:
sents = """それを<b>一つ</b>ください。
私は学生だ。
ソフトクリームを<b>二つ</b>ください。
カレーライスは700<b>円</b>です。
これはかなり<b>金</b>がかかった。
<b>これ</b>をください。
<b>水曜日</b>はバイトがあります。
<b>あれ</b>は何ですか。
お<b>先</b>にどうぞ。
<b>八日</b>からイギリスに行きます。
<b>そば</b>にいて下さい。
<b>こっち</b>に来て下さい。
<b>右</b>のポケットにハンカチが入っています。
この本、あなたに<b>あげます</b>。
<b>ここ</b>に本があります。
この本は<b>とても</b>おもしろい。
バナナが<b>七</b>本あります。
分かった人は<b>手</b>を上げてください。
<b>おなか</b>が空きました。
レストランは<b>空いていました</b>。"""
sents = re.sub("<b>(.*?)</b>", r"\1", sents).splitlines()

In [13]:
def pos_interpreter(pos):
    translations = {
        "名詞": "noun",
        "一般": "general",
        "助詞": "particle",
        "格助詞": "case-marking particle",
        "動詞": "verb",
        "自立": "independent",
        "助動詞": "auxilary verb",
        "記号": "symbol",
        "句点": "period",
        "代名詞": "pronoun",
        "係助詞": "binding particle",
        "数": "number",
        "接尾": "suffix",
        "助数詞": "counter"
    }
    tags = pos.split(',')
    real_tags = []
    for tag in tags:
        if tag != "*":
            if tag in translations.keys():
                real_tags.append(translations[tag])
            else:
                print(tag)
    return real_tags
print(pos_interpreter("助動詞,*,*,*"))

['auxilary verb']


In [14]:
from kuromojipy.kuromoji_server import KuromojiServer
kuro_server = KuromojiServer()
def pos_tag(sentence):
    tagged_words = []
    kuromoji = kuro_server.kuromoji
    tokenizer = kuromoji.Tokenizer.builder().build()
    tokens = tokenizer.tokenize(sentence)
    for elem in [[x.getBaseForm(), pos_interpreter(x.getPartOfSpeech())] for x in tokens]:
        tagged_words.append(elem)
    return tagged_words


In [15]:
pos_tag(sents[1])

[['私', ['noun', 'pronoun', 'general']],
 ['は', ['particle', 'binding particle']],
 ['学生', ['noun', 'general']],
 ['だ', ['auxilary verb']],
 ['。', ['symbol', 'period']]]

In [16]:
def encode_tags(tags):
    tag_encodings = {
        "noun": 1,
        "general": 2,
        "particle": 3,
        "case-marking particle": 4,
        "verb": 5,
        "independent": 6,
        "auxilary verb": 7,
        "symbol": 8,
        "period": 9,
        "pronoun": 10,
        "binding particle": 11,
        "number": 12,
        "suffix": 13,
        "counter": 14,
    }
    encoded = ""
    for tag in tags:
        encoded += "".join([
            "ő", 
            tag[0], 
            ";", 
            ",".join([str(tag_encodings[x]) for x in tag[1]]),
            "ú"
        ])
    return encoded
encode_tags(pos_tag(sents[0]))

'őそれ;1,10,2úőを;3,4,2úő一つ;1,2úőくださる;5,6úő。;8,9ú'

In [17]:
print(pos_tag(sents[3]))

[['カレーライス', ['noun', 'general']], ['は', ['particle', 'binding particle']], [None, ['noun', 'number']], ['円', ['noun', 'suffix', 'counter']], ['です', ['auxilary verb']], ['。', ['symbol', 'period']]]


In [18]:
def recognize_da(encoded):
    if re.search("ő.+?;1.*?úőだ;7ú", encoded) is not None:
        return "Declaring using だ"

def recognize_grammar(sent):
    tagged = pos_tag(sent)
    encoded = encode_tags(tagged)
    grammars = [recognize_da]
    found = [x(encoded) for x in grammars]
    found = [x for x in found if x is not None]
    return found

print(recognize_grammar(sents[3]))
    

TypeError: sequence item 1: expected str instance, NoneType found